<a href="https://colab.research.google.com/github/chh172/max-cut/blob/main/max_cut_in_Pointer%26Actor_Critic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
keras.__version__

'2.8.0'

In [14]:
import random 
import tensorflow as tf
from networkx import *
n = 6
p = 0.5
g = erdos_renyi_graph(n, p)
print(g. nodes)
# [0, 1, 2, 3, 4, 5] 
print(g. edges)
# [(0, 1), (0, 2), (0, 4), (1, 2), (1, 5), (3, 4), (4, 5)]
vertices = 5
A = adjacency_matrix(erdos_renyi_graph(vertices, 0.5)).todense()
A = tf.cast(A,tf.float32)
print(A)
#B = tf.concat([A,tf.zeros([1,5])],0)
#print(B)
#C = tf.concat([B,tf.zeros([6,1])],1)
#print(C)
#D = adjacency_matrix(erdos_renyi_graph(vertices, 0.5)).todense()
#D = tf.cast(D,tf.float32)
#print(D)
#print(A-D)
#print(tf.norm(A-D))
#print (1<2)
print(A[0][4])

[0, 1, 2, 3, 4, 5]
[(0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (3, 4), (4, 5)]
tf.Tensor(
[[0. 1. 1. 1. 1.]
 [1. 0. 1. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 0.]], shape=(5, 5), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [1]:
import numpy as np
from networkx import *
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import keras.backend as K
# from keras.engine import InputSpec
from tensorflow.keras.layers import InputSpec
from keras.activations import tanh, softmax
from keras.layers import LSTM
from keras.layers import Dense

# copyright to https://github.com/keon/pointer-networks
class Attention(keras.layers.Layer):
    """
        Attention layer
    """

    def __init__(self, hidden_dimensions, name='attention'):
        super(Attention, self).__init__(name=name, trainable=True)
        self.W1 = keras.layers.Dense(hidden_dimensions, use_bias=False)
        self.W2 = keras.layers.Dense(hidden_dimensions, use_bias=False)
        self.V = keras.layers.Dense(1, use_bias=False)

    def call(self, encoder_outputs, dec_output, mask=None):

        w1_e = self.W1(encoder_outputs)
        w2_d = self.W2(dec_output)
        tanh_output = tanh(w1_e + w2_d)
        v_dot_tanh = self.V(tanh_output)
        #print(v_dot_tanh.shape)
        if mask is not None:
            v_dot_tanh += (mask * -1e9)
        attention_weights = softmax(v_dot_tanh, axis=1)
        att_shape = K.shape(attention_weights)
        return K.reshape(attention_weights, (att_shape[0], att_shape[1]))


class Decoder(keras.layers.Layer):
    """
        Decoder class for PointerLayer
    """

    def __init__(self, hidden_dimensions):
        super(Decoder, self).__init__()
        self.lstm = keras.layers.LSTM(
            hidden_dimensions, return_sequences=False, return_state=True)

    def call(self, x, hidden_states):
        dec_output, state_h, state_c = self.lstm(
            x, initial_state=hidden_states)
        return dec_output, [state_h, state_c]

    def get_initial_state(self, inputs):
        return self.lstm.get_initial_state(inputs)

    def process_inputs(self, x_input, initial_states, constants):
        return self.lstm._process_inputs(x_input, initial_states, constants)


class PointerLSTM(keras.layers.Layer):
    """
        PointerLSTM
    """

    def __init__(self, hidden_dimensions, name='pointer', **kwargs):
        super(PointerLSTM, self).__init__(**kwargs)
        self.hidden_dimensions = hidden_dimensions
        self.attention = Attention(hidden_dimensions)
        self.decoder = Decoder(hidden_dimensions)

    def build(self, input_shape):
        super(PointerLSTM, self).build(input_shape)
        self.input_spec = [InputSpec(shape=input_shape)]

    def call(self, x, training=None, mask=None, states=None):
        """
        :param Tensor x: Should be the output of the decoder
        :param Tensor states: last state of the decoder
        :param Tensor mask: The mask to apply
        :return: Pointers probabilities
        """

        input_shape = self.input_spec[0].shape
        en_seq = x
        x_input = x[:, input_shape[1] - 1, :]
        x_input = K.repeat(x_input, input_shape[1])
        if states:
            initial_states = states
        else:
            initial_states = self.decoder.get_initial_state(x_input)

        constants = []
        preprocessed_input, _, constants = self.decoder.process_inputs(
            x_input, initial_states, constants)
        constants.append(en_seq)
        last_output, outputs, states = K.rnn(self.step, preprocessed_input,
                                             initial_states,
                                             go_backwards=self.decoder.lstm.go_backwards,
                                             constants=constants,
                                             input_length=input_shape[1])

        return outputs

    def step(self, x_input, states):
        x_input = K.expand_dims(x_input,1)
        input_shape = self.input_spec[0].shape
        en_seq = states[-1]
        _, [h, c] = self.decoder(x_input, states[:-1])
        dec_seq = K.repeat(h, input_shape[1])
        probs = self.attention(dec_seq, en_seq)
        return probs, [h, c]

    def get_output_shape_for(self, input_shape):
        # output shape is not affected by the attention component
        return (input_shape[0], input_shape[1], input_shape[1])

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], input_shape[1])

In [2]:
# Description: this function randomly generate a sequence of graphs G(V,E)
# where graphs are represented by their adjacency matrix A
# and the distribution of V and E are all uniform.

import numpy as np
from networkx import *
import random
def graph_batch_generator(batch, vertices):
  atlas = []
  for i in range(batch):
    A = adjacency_matrix(erdos_renyi_graph(vertices, 0.5)).todense()
    B = tf.concat([A,tf.zeros([1,vertices])],0)
    C = tf.concat([B,tf.zeros([vertices+1,1])],1)
    atlas.append(C)
  return tf.cast(np.array(atlas),tf.float32)   

graphs = graph_batch_generator(2, 5)
print(graphs)

tf.Tensor(
[[[0. 1. 0. 0. 1. 0.]
  [1. 0. 0. 1. 1. 0.]
  [0. 0. 0. 1. 0. 0.]
  [0. 1. 1. 0. 1. 0.]
  [1. 1. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0.]
  [1. 0. 1. 1. 1. 0.]
  [0. 1. 0. 1. 0. 0.]
  [0. 1. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]], shape=(2, 6, 6), dtype=float32)


In [ ]:
# Description: this function randomly generate a random graph G(V,E)
# where graph is represented by their adjacency matrix A, plus 'Split'

def graph_generator(vertices):
  A = adjacency_matrix(erdos_renyi_graph(vertices, 0.5)).todense()
  B = tf.concat([A,tf.zeros([1,vertices])],0)
  C = tf.concat([B,tf.zeros([vertices+1,1])],1)
  return tf.cast(np.array([C]),tf.float32)

print(graph_generator(5))

In [ ]:
# Unit testing
# atlas = graph_seq_generator(4,1,5)
#print(atlas[1])
#inp = Input(atlas[1])
#print(inp)
#for i in range(3):
  #print(g[i].nodes)
  #
  #print(g[i].edges)
  #
  #print(len(g[i].nodes))
#g = random.sample(atlas,2)
#print(g[1])
#input = input_generator(atlas)
#input_2 = tf.cast(np.array([atlas[1]]),tf.float32)
#print(input)
#print(input_2)
#print(input.shape[0])
#print(input.shape[1])
#print(input.shape[2])
#print(input.shape)
#print(input.shape)
# x = Input(shape=(32,))
#x_1 = Input(input)
# print(x)
# print(graph_generator(5))
# print(tf.keras.Input(shape=(10,None,5),tensor=graph_generator(5)))
def increment(x):
  x = x +1
  return
x = 1
increment(x)
print(x)  

1


In [ ]:

from keras.layers import Dense
# training/inference main in way 2

hidden_size = 128
vertices_size = 5
batch = 2
# input is a [A], where A is the adjacency matrix, which is a square matrix
# sequence length shall match the number of rows and feature_dim is number of cols
# note the feature for each node is its adjacency relation (row vector in adjacency matrix)
seq_len = vertices_size+1
feature_dim = vertices_size+1

encoder = LSTM(hidden_size,return_sequences = True, name="encoder",return_state=True)
actor_decoder = PointerLSTM( hidden_size, name="actor_decoder")
critic_decoder = LSTM(hidden_size,name="critic_decoder")
value_f = Dense(1)

inputs = keras.layers.Input(shape=(seq_len, feature_dim)) 
encoder_o, state_h, state_c = encoder(inputs)
policy = actor_decoder(encoder_o,states=[state_h, state_c])
#scores = critic_decoder(encoder_o)
decoder_o = critic_decoder(encoder_o)
scores = value_f(decoder_o)

model = tf.keras.Model(inputs=inputs, outputs=[policy,scores])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.Huber(),
              metrics=['accuracy'])

graphs = graph_batch_generator(batch,vertices_size)
print(graphs)
#encoder_outputs, state_h, state_c = encoder(graphs)
#print(encoder_outputs)
#print("hello world")
#policy = actor_decoder(encoder_outputs)
#print(policy)
#decoder_o = critic_decoder(encoder_outputs)
#scores  = value_f(decoder_o)
print("hello world")
#print(scores)

#print(model.predict(graphs)[1][0])

In [12]:
def reward(state,action):
  return 1.0

def value_f_update(critic, delta ,phi, beta):
  return phi+ beta*delta*value_grad(critic,phi)

def policy_update(actor,alpha,lamb,delta,theta):
  return theta + alpha*lamb*delta*policy_grad(actor,theta)
  
def policy_grad(actor,graphs):
  with tf.GradientTape() as tape:
    tape.watch(graphs)
    y = tf.math.log(actor(graphs))
  return tape.gradient(y,actor.trainable_weights) 
def value_grad(critic,graphs):
  with tf.GradientTape() as tape:
    tape.watch(graphs)
    y = critic(graphs)
  return tape.gradient(y, critic.trainable_weights) 
# MDP construction

# 'State space, Action space, and Transition':
# In our MDP, a graph as a whole is a state and each graph/non-terminal 
# state is associated with its characteristic action space A_s
# i.e. the whole action action space A = U_s A_s partitioned by each state.
# Moreover, we define a uniform terminal state s_t, i.e. p(s,a_s) := s_t for 
# any state s and its accompanied action a_s, where p is the transition function.

# 'Reward':
# Reward r of the action a_s from the state/graph s is defined to be 
# the cut-value produced by the cut of s under action a_s, 
# i.e. r(s,a_s) := cut_Value(s^{a_s}), 
# where s^{a_s} is the cut of s under action a_s. 



In [ ]:
from keras.layers import Dense
# training main in way 2

# constants, alpha,beta learning rate
disc_rate = 0.95
alpha = 0.01
beta = 0.01

# model construction
def model_constr(vertices_size, hidden_size):

  # input is a [A], where A is the adjacency matrix, which is a square matrix
  # sequence length shall match the number of rows and feature_dim is number of cols
  # note the feature for each node is its adjacency relation (row vector in adjacency matrix)
  seq_len = vertices_size+1
  feature_dim = vertices_size+1

  encoder = LSTM(hidden_size, trainable=True,return_sequences = True, name="encoder",return_state=True)
  actor_decoder = PointerLSTM(hidden_size, trainable=True, name="actor_decoder")
  critic_decoder = LSTM(hidden_size, trainable=True,name="critic_decoder")
  value_f = Dense(1)

  inputs = keras.layers.Input(shape=(seq_len, feature_dim)) 
  encoder_o, state_h, state_c = encoder(inputs)
  policy = actor_decoder(encoder_o,states=[state_h, state_c])
  decoder_o = critic_decoder(encoder_o)
  scores = value_f(decoder_o)

  actor = tf.keras.Model(inputs=inputs, outputs=policy)
  critic = tf.keras.Model(inputs=inputs, outputs=scores)
  actor.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                loss=tf.keras.losses.Huber(),
                metrics=['accuracy'])
  critic.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                loss=tf.keras.losses.Huber(),
                metrics=['accuracy'])
  # model = tf.keras.Model(inputs=inputs, outputs=[policy,scores])
  # model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              # loss=tf.keras.losses.Huber(),
              # metrics=['accuracy'])


  return [actor,critic]

# MDP construction

# 'State space, Action space, and Transition':
# In our MDP, a graph as a whole is a state and each graph/non-terminal 
# state is associated with its characteristic action space A_s
# i.e. the whole action action space A = U_s A_s partitioned by each state.
# Moreover, we define a uniform terminal state s_t, i.e. p(s,a_s) := s_t for 
# any state s and its accompanied action a_s, where p is the transition function.

# 'Reward':
# Reward r of the action a_s from the state/graph s is defined to be 
# the cut-value produced by the cut of s under action a_s, 
# i.e. r(s,a_s) := cut_Value(s^{a_s}), 
# where s^{a_s} is the cut of s under action a_s. 



# actor-critic training
def training(batch, vertices_size, hidden_size):
  graphs = graph_batch_generator(batch,vertices_size)
  actor, critic = model_constr(vertices_size, hidden_size)

  # randomly init 'phi', 'theta' (already set)

  lamb = 1
    # do until theta converges (loop for every graph)
  converge = False
  for i in range(batch):
    # randomly init 'phi', 'theta' (already set)
    phi = critic.trainable_weights
    theta = actor.trainable_weights
    # init starting state
    s = graphs[i]
    # s virtually transitioned to terminal, but in fact stay the same till convergence
    while not converge :
      # In state 's', select action 'a' given by actor
      a = actor.predict(graphs)[i]
      # perform 'a' and collect reward r and new state (terminal state)
      r = reward(s,a)        
      # update delta
      # print(critic.predict(graphs)[i].dtype)
      delta = r - critic.predict(graphs)[i] 
      # update value function parameter 'phi'
      phi = value_f_update(critic, delta ,phi, beta)
      critic.set_weights(phi)
      # update policy parameter 'theta'
      theta_p = policy_update(actor,alpha,lamb,delta,theta)
      # update loop condition
      converge = tf.norm(theta - theta_p) < 0.5
      theta = theta_p
      # update discount
      lamb = lamb * disc_rate 
  return [actor, critic]

batch = 2
vertices_size = 3
hidden_size = 4
#graphs = graph_batch_generator(batch,vertices_size)
#print(graphs)
#actor, critic = model_constr(vertices_size, hidden_size)
print("hello world")
#print(scores)
# print(model.trainable_weights)
#print(actor.predict(graphs))
#print(critic.predict(graphs))
# print(batch)
actor, critic = training(batch, vertices_size, hidden_size)

In [14]:
batch = 2
vertices_size = 3
hidden_size = 4
graphs = graph_batch_generator(batch,vertices_size)
actor, critic = model_constr(vertices_size, hidden_size)
print(actor.predict(graphs))
print(critic.predict(graphs))

[[[0.253188   0.25789192 0.24348596 0.24543417]
  [0.25319415 0.25787926 0.24348995 0.24543662]
  [0.25319743 0.25787112 0.24349274 0.24543874]
  [0.253199   0.25786665 0.24349438 0.24544004]]

 [[0.25150052 0.25787047 0.24460709 0.24602196]
  [0.2515133  0.2578734  0.24459597 0.2460174 ]
  [0.25152126 0.25787464 0.24458958 0.24601452]
  [0.25152603 0.25787532 0.244586   0.24601269]]]
[[-0.05615778]
 [-0.01478709]]
